In [6]:
import QuantLib as ql
import numpy as np
from timeit import default_timer as timer

# option data
calculation_date = ql.Date(27, 10, 2019)
maturity_date = ql.Date(30, 10, 2020)
stock_price = ql.SimpleQuote(150.0)
strike_price = 150.0
volatility = ql.SimpleQuote(0.10) # the historical vols for a year
dividend_rate =  ql.SimpleQuote(0.002)
risk_free_rate = ql.SimpleQuote(0.003)
option_type = ql.Option.Put
barrier = 160.0
barrier_type = ql.Barrier.UpOut
rebate = 0.0

day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

ql.Settings.instance().evaluationDate = calculation_date

# construct the European Option
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_handle = ql.QuoteHandle(stock_price)
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, ql.QuoteHandle(risk_free_rate), day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, ql.QuoteHandle(dividend_rate), day_count))
flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, ql.QuoteHandle(volatility), day_count))
bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_ts, 
                                           flat_vol_ts)


In [7]:
option = ql.BarrierOption(barrier_type, barrier, rebate, 
                       payoff, 
                       ql.EuropeanExercise(maturity_date))
option.setPricingEngine(ql.AnalyticBarrierEngine(bsm_process))
trueValue = option.NPV()

print("The theoretical price is ", trueValue)

The theoretical price is  5.2214083456793166


In [8]:
#variable names need to be changed

class BarrierOption:
    def __init__(self, calculation_date, maturity, stock_price, strike_price, volatility, dividend_rate, risk_free_rate, option_type, barrier, barrier_type, rebate):
        self.maturity = maturity
        self.stock_price = stock_price
        self.strike_price = strike_price
        self.volatility = volatility
        self.dividend_rate = dividend_rate
        self.risk_free_rate = risk_free_rate
        self.option_type = option_type
        self.calculation_date = calculation_date
        self.barrier = barrier
        self.barrier_type = barrier_type
        self.rebate = rebate
        self.bs_price = -1
        
        
    def BSM_price(self):
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()
        ql.Settings.instance().evaluationDate = self.calculation_date
        
        payoff = ql.PlainVanillaPayoff(self.option_type, self.strike_price)
        exercise = ql.EuropeanExercise(self.maturity)
        european_option = ql.VanillaOption(payoff, exercise)
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(self.stock_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, ql.QuoteHandle(ql.SimpleQuote(self.risk_free_rate)), day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, ql.QuoteHandle(ql.SimpleQuote(self.dividend_rate)), day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(self.calculation_date, calendar, ql.QuoteHandle(ql.SimpleQuote(self.volatility)), day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                                   dividend_yield, 
                                                   flat_ts, 
                                                   flat_vol_ts)
        option = ql.BarrierOption(self.barrier_type, self.barrier, self.rebate, 
                       payoff, exercise)
        option.setPricingEngine(ql.AnalyticBarrierEngine(bsm_process))
        trueValue = option.NPV()
        self.bs_price = trueValue
        return self.bs_price
    
    def data_set(self):
        '''
        Funtion to return a set of required data for one sample for training purpose.
        
        '''
        if self.bs_price == -1:
            self.BSM_price()
        maturity_in_year = (self.maturity - self.calculation_date)/365
        data_set = (self.stock_price, self.strike_price,self.barrier, maturity_in_year, self.dividend_rate, self.volatility, self.risk_free_rate, self.rebate, self.bs_price)
        return data_set

In [9]:
stock_price = 150.0
barrier_price = 160.0
rebate = 0.0
volatility = 0.1
risk_free_rate = 0.003
dividend_rate = 0.002
barrier = BarrierOption(calculation_date, maturity_date, stock_price, strike_price, volatility, 
                        dividend_rate, risk_free_rate, option_type, barrier_price, barrier_type, rebate
                        )
print('barrier price: ', barrier.BSM_price())

barrier price:  5.2214083456793166


In [10]:
import datetime
import random
import pandas as pd

'''Date helper functions'''
def xldate_to_datetime(xldate):
    temp = datetime.datetime(1899, 12, 30)
    delta = datetime.timedelta(days=xldate)
    return temp+delta

def ql_to_datetime(d):
    return datetime.datetime(d.year(), d.month(), d.dayOfMonth())

def datetime_to_xldate(date):
    temp = datetime.datetime(1899, 12, 30)
    return (date - temp).days

def random_barrier_options_pd(numbers = 0):
    options = []
    start_maturity = datetime.datetime(2020,11,1)
    end_maturity = datetime.datetime(2023,10,30)
    calculation_date = datetime.datetime(2020,10,30)
    
    xldate1 = datetime_to_xldate(start_maturity)
    xldate2 = datetime_to_xldate(end_maturity)
    calculation_xldate = datetime_to_xldate(calculation_date)
    calculation_date = ql.Date(calculation_xldate)
    for number in range(numbers):
        maturity = ql.Date(random.randint(xldate1, xldate2+1))
        stock_price = random.randint(100, 501)
        strike_price = random.randint(7, 651)
        volatility = random.uniform(0.05, 0.90)
        dividend_rate = random.uniform(0, 0.003)
        risk_free_rate = random.uniform(0.001, 0.003)
        option_type = ql.Option.Put
        barrier_price = stock_price * (random.uniform(1, 1.2))
        barrier_type = ql.Barrier.UpOut
        rebate = 0.0
        option = BarrierOption(calculation_date, maturity, stock_price, strike_price, volatility, 
                        dividend_rate, risk_free_rate, option_type, barrier_price, barrier_type, rebate
                        )
        options.append(option.data_set())  
    dataframe = pd.DataFrame(options)
    dataframe.columns = ['stock_price', 'strike_price', 'barrier_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate', 'rebate', 'put_price']
    return dataframe

print(random_barrier_options_pd(1))


   stock_price  strike_price  barrier_price  maturity  devidends  volatility  \
0          398           539     476.946881  2.093151   0.002602    0.516766   

   risk_free_rate  rebate  put_price  
0        0.002039     0.0  95.444622  


In [11]:
n = 10000
df = random_barrier_options_pd(n)
## Normalize the data exploiting the fact that the BS Model is linear homogenous in S,K
df['stock_price'] = df['stock_price']/df['strike_price']
df['barrier_price'] = df['barrier_price']/df['strike_price']
df['put_price'] = df['put_price']/df['strike_price']
#print(df)
#print(df.isnull().any())
n_train =  (int)(0.8 * n)
train = df[0:n_train]
X_train = train[['stock_price', 'strike_price', 'barrier_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate']].values
y_train = train['put_price'].values
test = df[n_train+1:n]
X_test = test[['stock_price', 'strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate']].values
y_test = test['put_price'].values

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.models import Sequential
from keras import backend
import keras
import numpy
import os

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_weights.h5")
print("Loaded model from disk")
loaded_model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1)
]

# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])
loaded_model.fit(X_train, y_train, epochs=30, validation_split=0.2,callbacks=callbacks,
        shuffle=True, verbose=2)
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f" % (loaded_model.metrics_names[1], score[0]))

Loaded model from disk
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 256)               1792      
_________________________________________________________________
dropout_45 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_45 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_46 (Activation)   (

ValueError: Error when checking input: expected dense_56_input to have shape (6,) but got array with shape (7,)

In [97]:
#barrier options

from QuantLib import *
import matplotlib.pyplot as plt
stock_price = ql.SimpleQuote(150.0)
strike_price = 150
barrier, barrierType, optionType, rebate = (160, Barrier.UpOut, Option.Put, 0.0)
underlying, strike, rf, sigma, maturity, divYield = (10, 150, 0.003, 0.1, 12, 0.002)

## maturity is in days and must be an integer 
barrier_data = dict( (name,eval(name)) for name in [
    'barrier', 'barrierType', 'optionType', 'rebate', 'underlying', 'strike', 
    'rf', 'sigma', 'maturity', 'divYield'] )
barrier_data['barrierType'] = ['DownIn', 'UpIn', 'DownOut', 'UpOut'][barrier_data['barrierType']]
barrier_data['optionType'] = ['Put', '???', 'Call'][barrier_data['optionType']+1]
for k in barrier_data.keys():
    print("{} = {}".format(k, barrier_data[k]))

ql.Settings.instance().evaluationDate = calculation_date
maturity_date = ql.Date(30, 10, 2020)

process = BlackScholesMertonProcess(
    QuoteHandle(SimpleQuote(underlying)),
    YieldTermStructureHandle(FlatForward(calculation_date, divYield, Thirty360())),
    YieldTermStructureHandle(FlatForward(calculation_date, rf, Thirty360())),
    BlackVolTermStructureHandle(BlackConstantVol(
        calculation_date, NullCalendar(), sigma, Thirty360())))
option = BarrierOption(barrierType, barrier, rebate, 
                       PlainVanillaPayoff(optionType, strike), 
                       EuropeanExercise(maturity_date))
option.setPricingEngine(AnalyticBarrierEngine(process))
trueValue = option.NPV()

print(trueValue)

barrier = 160
barrierType = UpOut
optionType = Put
rebate = 0.0
underlying = 10
strike = 150
rf = 0.003
sigma = 0.1
maturity = 12
divYield = 0.002
139.56708195098898
